## Recommandation par profile d'acheteur (Gros acheteur ponctuel VS petit acheteur de tous les jours) 

### 1ere Etape: calculer nbr item par panier et total panier pour chaque client
### 2me Etape: calculer nbr item par panier  par client et total panier par client - pour chaque mail
### 3eme Etape: calcul cluster client
### 4eme Etape: trouver le cluster depuis un id client
### 5eme Etape: trouver quel mail fait parti de se cluster
### 6eme Recommandation dans la mail